In [ ]:
#Import libraries 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Paste player links for the 2010s Hall of Fame class
player_urls = [
    "https://www.basketball-reference.com/players/b/braunca01.html", #Carl Braun 2019
    "https://www.basketball-reference.com/players/c/coopech01.html", #Chuck Cooper 2019
    "https://www.basketball-reference.com/players/d/divacvl01.html", #Vlade Divac 2019
    "https://www.basketball-reference.com/players/j/jonesbo01.html", #Bobby Jones 2019
    "https://www.basketball-reference.com/players/m/moncrsi01.html", #Sidney Moncrief 2019
    "https://www.basketball-reference.com/players/s/sikmaja01.html", #Jack Sikma 2019
    "https://www.basketball-reference.com/players/w/westppa01.html", #Paul Westphal 2019
    "https://www.basketball-reference.com/players/a/allenra02.html", #Ray Allen 2018
    "https://www.basketball-reference.com/players/c/cheekma01.html", #Maurice Cheeks 2018
    "https://www.basketball-reference.com/players/h/hillgr01.html", #Grant Hill 2018
    "https://www.basketball-reference.com/players/k/kiddja01.html", #Jason Kidd 2018
    "https://www.basketball-reference.com/players/n/nashst01.html", #Steve Nash 2018
    "https://www.basketball-reference.com/players/r/radjadi01.html", #Dino Radja 2018
    "https://www.basketball-reference.com/players/s/scottch01.html", #Charlie Scott 2018
    "https://www.basketball-reference.com/players/m/mcginge01.html", #George McGinnis 2017
    "https://www.basketball-reference.com/players/m/mcgratr01.html", #Tracy McGrady 2017
    "https://www.basketball-reference.com/players/b/beatyze01.html", #Zelmo Beaty 2016
    "https://www.basketball-reference.com/players/i/iversal01.html", #Allen Iverson 2016
    "https://www.basketball-reference.com/players/m/mingya01.html", #Yao Ming 2016
    "https://www.basketball-reference.com/players/o/onealsh01.html", #Shaq 2016
    "https://www.basketball-reference.com/players/d/dampilo01.html", #Louie Dampier 2015
    "https://www.basketball-reference.com/players/h/haywosp01.html", #Spencer Haywood 2015
    "https://www.basketball-reference.com/players/m/mutomdi01.html", #Dikembe Mutombo 2015
    "https://www.basketball-reference.com/players/w/whitejo01.html", #Jojo White 2015
    "https://www.basketball-reference.com/players/m/marcisa01.html", #Sarunas Marciulionis 2014
    "https://www.basketball-reference.com/players/m/mournal01.html", #Alonzo Mourning 2014
    "https://www.basketball-reference.com/players/r/richmmi01.html", #Mitch Richmond 2014
    "https://www.basketball-reference.com/players/r/rodgegu01.html", #Guy Rodgers 2014
    "https://www.basketball-reference.com/players/b/brownro01.html", #Roger Brown 2013
    "https://www.basketball-reference.com/players/g/gueriri01.html", #Richie Geurin 2013
    "https://www.basketball-reference.com/players/k/kingbe01.html", #Bernard King 2013
    "https://www.basketball-reference.com/players/p/paytoga01.html" #Gary Payton 2013
]

#Headers for the request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

#Initialize a list to store all players' data
all_players_data = []

#Player position abbreviations
position_map = {
    'Center': 'C',
    'Power Forward': 'PF',
    'Small Forward': 'SF',
    'Shooting Guard': 'SG',
    'Point Guard': 'PG'
}

#Function to clean and extract player position
def get_position(soup):
    position = None  #Default position if not found
    p_elems = soup.find_all('p')

    for p_elem in p_elems:
        if 'Position:' in p_elem.get_text():
            position_text = p_elem.get_text(separator=" ").split("Position:")[1].strip()
            position_text = position_text.replace('▪', '').strip()
            if 'Shoots:' in position_text:
                position_text = position_text.split('Shoots:')[0].strip()
            position_text = " ".join(position_text.split())
            position_text = position_text.replace(' and ', ', ')
            positions = position_text.split(',')
            primary_position = positions[0].strip()
            position = position_map.get(primary_position, primary_position)
            break
    return position



#Function to safely extract MVP count
def get_mvp_count(soup):
    mvp_count = 0
    mvp_elem = soup.find_all('li', {'class': 'poptip'}, string=lambda s: s and 'MVP' in s and 'Finals' not in s and 'AS' not in s and 'MBWA NBA' not in s)
    for elem in mvp_elem:
        text = elem.text.strip()
        if 'x' in text:
            mvp_count += int(text.split('x')[0])  #For players with multiple MVPs, split on 'x' and extract the first element
        else:
            mvp_count += 1  #For players with only 1 MVP
    return mvp_count

#Function to safely extract Scoring Championships count
def get_scoring_champ_count(soup):
    scoring_champ_count = 0
    scoring_champ_elem = soup.find_all('li', {'data-tip': lambda x: x and 'NBA Scoring Champ' in x})
    for elem in scoring_champ_elem:
        text = elem.text.strip()
        if 'x' in text:
            scoring_champ_count += int(text.split('x')[0])  # For players with multiple scoring championships, split on 'x'
        else:
            scoring_champ_count += 1  # For players with just 1 scoring championship
    return scoring_champ_count

#Function to safely extract NBA Championships count
def get_chips_count(soup):
    chips_count = 0
    chip_elem = soup.find_all('li', class_='', string=lambda s: s and 'NBA Champ' in s)
    for elem in chip_elem:
        text = elem.text.strip()
        if 'x' in text:
            chips_count += int(text.split('x')[0])  #For players with multiple championships, split on 'x'
        else:
            chips_count += 1  #For players with 1 championship
    return chips_count

#Loop over each player URL to scrape their data
for url in player_urls:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            #Get player name
            player_name = soup.find('h1').find('span').text.strip()

            #Get player position
            position = get_position(soup)

            #Safeguard function for extracting stats
            def safe_find(tag, text):
                element = soup.find('span', {'data-tip': text})
                if element:
                    return element.find_next('p').find_next('p').text.strip()
                return None

            games = int(safe_find('Games', 'Games'))
            ppg = float(safe_find('Points', 'Points'))
            rpg = float(safe_find('Total Rebounds', 'Total Rebounds'))
            apg = float(safe_find('Assists', 'Assists'))

            #Extract PER
            per_elem = soup.find('span', {'data-tip': lambda x: x and 'Player Efficiency Rating' in x})
            per = float(per_elem.find_next('p').find_next('p').text.strip() if per_elem else None)

            #Extract Field Goal Percentage and Free Throw Percentage
            fg_pct = float(safe_find('Field Goal Percentage', 'Field Goal Percentage'))
            ft_pct = float(safe_find('Free Throw Percentage', 'Free Throw Percentage'))

            #Extract awards counts
            mvp_count = get_mvp_count(soup)
            scoring_champ_count = get_scoring_champ_count(soup)
            chips_count = get_chips_count(soup)

            #Extract All-Stars, All-NBA, All-Defense, and other honors
            all_stars = int(soup.find('li', {'class': 'all_star'}).find('a').text.strip().split('x')[0] if soup.find('li', {'class': 'all_star'}) else 0)
            all_nba_count = sum([int(a.text.strip().split('x')[0]) if 'x' in a.text else 1 for a in soup.find('li', string=lambda s: s and 'All-NBA' in s).find('a')]) if soup.find('li', string=lambda s: s and 'All-NBA' in s) else 0
            all_defense_count = sum([int(a.text.strip().split('x')[0]) if 'x' in a.text else 1 for a in soup.find('li', {'class': 'poptip'}, string=lambda s: s and 'All-Defensive' in s).find('a')]) if soup.find_all('li', {'class': 'poptip'}, string=lambda s: s and 'All-Defensive' in s) else 0
            all_rookie = 1 if soup.find('li', {'data-tip': lambda s: s and 'All-Rookie' in s}) else 0
            roy = 1 if soup.find('li', {'class': 'poptip'}, string=lambda s: s and 'ROY' in s) else 0
            dpoy_count = sum([int(text.split('x')[0]) if 'x' in text else 1 for text in [elem.text.strip() for elem in soup.find_all('li', class_='poptip', string=lambda s: s and 'Def. POY' in s)]])
            
            #All players are in the Hall of Fame
            hof = 1

            #Store the data
            player_data = {
                'Name': player_name,
                'Position': position,
                'Games': games,
                'PPG': ppg,
                'RPG': rpg,
                'APG': apg,
                'PER': per,
                'FG%': fg_pct,
                'FT%': ft_pct,
                'All-Stars': all_stars,
                'All-NBA': all_nba_count,
                'All-Defense': all_defense_count,
                'All-Rookie Team': all_rookie,
                'MVPs': mvp_count,
                'Chips': chips_count,
                'ROY': roy,
                'DPOYs': dpoy_count,
                'Scoring Champ': scoring_champ_count,
                'HOF': hof
            }
            all_players_data.append(player_data)

        except Exception as e:
            print(f"Error scraping data for {url}: {e}")

        time.sleep(1)  #Be polite with requests, avoid overwhelming the server

#Create a DataFrame from the collected data
df_2019_2013 = pd.DataFrame(all_players_data)


In [2]:
df_2019_2013

,Name,Position,Games,PPG,RPG,APG,PER,FG%,FT%,All-Stars,All-NBA,All-Defense,All-Rookie Team,MVPs,Chips,ROY,DPOYs,Scoring Champ,HOF
0,Carl Braun,PG,788,13.5,3.4,3.7,15.8,38.3,80.4,5,0,0,0,0,1,0,0,0,1
1,Chuck Cooper,SF,409,6.7,5.9,1.8,11.1,33.9,74.3,0,0,0,0,0,0,0,0,0,1
2,Vlade Divac,C,1134,11.8,8.2,3.1,17.7,49.5,69.2,1,0,0,1,0,0,0,0,0,1
3,Bobby Jones,PF,941,12.1,6.1,2.7,18.2,56.0,76.6,5,0,11,1,0,1,1,0,0,1
4,Sidney Moncrief,SG,767,15.6,4.7,3.6,18.7,50.2,83.1,5,5,5,0,0,0,0,2,0,1
5,Jack Sikma,C,1107,15.6,9.8,3.2,17.3,46.4,84.9,7,0,1,1,0,1,0,0,0,1
6,Paul Westphal,SG,823,15.6,1.9,4.4,19.4,50.4,82.0,5,4,0,0,0,1,0,0,0,1
7,Ray Allen,SG,1300,18.9,4.1,3.4,18.6,45.2,89.4,10,2,0,1,0,2,0,0,0,1
8,Maurice Cheeks,PG,1101,11.1,2.8,6.7,16.5,52.3,79.3,4,0,5,0,0,1,0,0,0,1
9,Grant Hill,SF,1026,16.7,6.0,4.1,19.0,48.3,76.9,7,5,0,1,0,0,1,0,0,1


In [3]:
#Import libraries 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Paste player links for the 2020s Hall of Fame class
player_urls = [
    "https://www.basketball-reference.com/players/d/danieme01.html", #Mel Daniels 2012
    "https://www.basketball-reference.com/players/m/millere01.html", #Reggie Miller 2012
    "https://www.basketball-reference.com/players/s/sampsra01.html", #Ralph Sampson 2012
    "https://www.basketball-reference.com/players/w/walkech01.html", #Chet Walker 2012
    "https://www.basketball-reference.com/players/w/wilkeja01.html", #Jamaal Wilkes 2012
    "https://www.basketball-reference.com/players/g/gilmoar01.html", #Artis Gilmore 2011
    "https://www.basketball-reference.com/players/m/mullich01.html", #Chris Mullin 2011
    "https://www.basketball-reference.com/players/r/rodmade01.html", #Dennis Rodman 2011
    "https://www.basketball-reference.com/players/s/sabonar01.html", #Arvydas Sabonis 2011
    "https://www.basketball-reference.com/players/j/johnsde01.html", #Dennis Johnson 2010
    "https://www.basketball-reference.com/players/j/johnsgu01.html", #Gus Johnson 2010
    "https://www.basketball-reference.com/players/m/malonka01.html", #Karl Malone 2010
    "https://www.basketball-reference.com/players/p/pippesc01.html" #Scottie Pippen 2010
]

#Headers for the request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

#Initialize a list to store all players' data
all_players_data = []

#Player position abbreviations
position_map = {
    'Center': 'C',
    'Power Forward': 'PF',
    'Small Forward': 'SF',
    'Shooting Guard': 'SG',
    'Point Guard': 'PG'
}

#Function to clean and extract player position
def get_position(soup):
    position = None  #Default position if not found
    p_elems = soup.find_all('p')

    for p_elem in p_elems:
        if 'Position:' in p_elem.get_text():
            position_text = p_elem.get_text(separator=" ").split("Position:")[1].strip()
            position_text = position_text.replace('▪', '').strip()
            if 'Shoots:' in position_text:
                position_text = position_text.split('Shoots:')[0].strip()
            position_text = " ".join(position_text.split())
            position_text = position_text.replace(' and ', ', ')
            positions = position_text.split(',')
            primary_position = positions[0].strip()
            position = position_map.get(primary_position, primary_position)
            break
    return position



#Function to safely extract MVP count
def get_mvp_count(soup):
    mvp_count = 0
    mvp_elem = soup.find_all('li', {'class': 'poptip'}, string=lambda s: s and 'MVP' in s and 'Finals' not in s and 'AS' not in s and 'MBWA NBA' not in s)
    for elem in mvp_elem:
        text = elem.text.strip()
        if 'x' in text:
            mvp_count += int(text.split('x')[0])  #For players with multiple MVPs, split on 'x' and extract the first element
        else:
            mvp_count += 1  #For players with only 1 MVP
    return mvp_count

#Function to safely extract Scoring Championships count
def get_scoring_champ_count(soup):
    scoring_champ_count = 0
    scoring_champ_elem = soup.find_all('li', {'data-tip': lambda x: x and 'NBA Scoring Champ' in x})
    for elem in scoring_champ_elem:
        text = elem.text.strip()
        if 'x' in text:
            scoring_champ_count += int(text.split('x')[0])  # For players with multiple scoring championships, split on 'x'
        else:
            scoring_champ_count += 1  # For players with just 1 scoring championship
    return scoring_champ_count

#Function to safely extract NBA Championships count
def get_chips_count(soup):
    chips_count = 0
    chip_elem = soup.find_all('li', class_='', string=lambda s: s and 'NBA Champ' in s)
    for elem in chip_elem:
        text = elem.text.strip()
        if 'x' in text:
            chips_count += int(text.split('x')[0])  #For players with multiple championships, split on 'x'
        else:
            chips_count += 1  #For players with 1 championship
    return chips_count

#Loop over each player URL to scrape their data
for url in player_urls:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            #Get player name
            player_name = soup.find('h1').find('span').text.strip()

            #Get player position
            position = get_position(soup)

            #Safeguard function for extracting stats
            def safe_find(tag, text):
                element = soup.find('span', {'data-tip': text})
                if element:
                    return element.find_next('p').find_next('p').text.strip()
                return None

            games = int(safe_find('Games', 'Games'))
            ppg = float(safe_find('Points', 'Points'))
            rpg = float(safe_find('Total Rebounds', 'Total Rebounds'))
            apg = float(safe_find('Assists', 'Assists'))

            #Extract PER
            per_elem = soup.find('span', {'data-tip': lambda x: x and 'Player Efficiency Rating' in x})
            per = float(per_elem.find_next('p').find_next('p').text.strip() if per_elem else None)

            #Extract Field Goal Percentage and Free Throw Percentage
            fg_pct = float(safe_find('Field Goal Percentage', 'Field Goal Percentage'))
            ft_pct = float(safe_find('Free Throw Percentage', 'Free Throw Percentage'))

            #Extract awards counts
            mvp_count = get_mvp_count(soup)
            scoring_champ_count = get_scoring_champ_count(soup)
            chips_count = get_chips_count(soup)

            #Extract All-Stars, All-NBA, All-Defense, and other honors
            all_stars = int(soup.find('li', {'class': 'all_star'}).find('a').text.strip().split('x')[0] if soup.find('li', {'class': 'all_star'}) else 0)
            all_nba_count = sum([int(a.text.strip().split('x')[0]) if 'x' in a.text else 1 for a in soup.find('li', string=lambda s: s and 'All-NBA' in s).find('a')]) if soup.find('li', string=lambda s: s and 'All-NBA' in s) else 0
            all_defense_count = sum([int(a.text.strip().split('x')[0]) if 'x' in a.text else 1 for a in soup.find('li', {'class': 'poptip'}, string=lambda s: s and 'All-Defensive' in s).find('a')]) if soup.find_all('li', {'class': 'poptip'}, string=lambda s: s and 'All-Defensive' in s) else 0
            all_rookie = 1 if soup.find('li', {'data-tip': lambda s: s and 'All-Rookie' in s}) else 0
            roy = 1 if soup.find('li', {'class': 'poptip'}, string=lambda s: s and 'ROY' in s) else 0
            dpoy_count = sum([int(text.split('x')[0]) if 'x' in text else 1 for text in [elem.text.strip() for elem in soup.find_all('li', class_='poptip', string=lambda s: s and 'Def. POY' in s)]])
            
            #All players are in the Hall of Fame
            hof = 1

            #Store the data
            player_data = {
                'Name': player_name,
                'Position': position,
                'Games': games,
                'PPG': ppg,
                'RPG': rpg,
                'APG': apg,
                'PER': per,
                'FG%': fg_pct,
                'FT%': ft_pct,
                'All-Stars': all_stars,
                'All-NBA': all_nba_count,
                'All-Defense': all_defense_count,
                'All-Rookie Team': all_rookie,
                'MVPs': mvp_count,
                'Chips': chips_count,
                'ROY': roy,
                'DPOYs': dpoy_count,
                'Scoring Champ': scoring_champ_count,
                'HOF': hof
            }
            all_players_data.append(player_data)

        except Exception as e:
            print(f"Error scraping data for {url}: {e}")

        time.sleep(1)  #Be polite with requests, avoid overwhelming the server

#Create a DataFrame from the collected data
df_2010_2012 = pd.DataFrame(all_players_data)

#Save to a CSV file
df_2010_2012.to_csv('2010-2012_hof_players.csv', index=False)

In [4]:
df_2010_2012

,Name,Position,Games,PPG,RPG,APG,PER,FG%,FT%,All-Stars,All-NBA,All-Defense,All-Rookie Team,MVPs,Chips,ROY,DPOYs,Scoring Champ,HOF
0,Mel Daniels,C,639,18.4,14.9,1.8,20.1,46.8,65.7,7,0,0,1,2,0,1,0,0,1
1,Reggie Miller,SG,1389,18.2,3.0,3.0,18.4,47.1,88.8,5,3,0,0,0,0,0,0,0,1
2,Ralph Sampson,C,456,15.4,8.8,2.3,16.0,48.6,66.1,4,1,0,1,0,0,1,0,0,1
3,Chet Walker,SF,1032,18.2,7.1,2.1,17.6,47.0,79.6,7,0,0,1,0,1,0,0,0,1
4,Jamaal Wilkes,SF,828,17.7,6.2,2.5,16.5,49.9,75.9,3,0,2,1,0,4,1,0,0,1
5,Artis Gilmore,C,1329,18.8,12.3,2.3,21.4,58.2,69.8,11,0,5,1,1,0,1,0,0,1
6,Chris Mullin,SF,986,18.2,4.1,3.5,18.8,50.9,86.5,5,4,0,0,0,0,0,0,0,1
7,Dennis Rodman,PF,911,7.3,13.1,1.8,14.6,52.1,58.4,2,2,8,0,0,5,0,2,0,1
8,Arvydas Sabonis,C,470,12.0,7.3,2.1,21.2,50.0,78.6,0,0,0,1,0,0,0,0,0,1
9,Dennis Johnson,PG,1100,14.1,3.9,5.0,14.6,44.5,79.7,5,2,9,0,0,3,0,0,0,1


In [3]:
import pandas as pd 
df_2010_2012 = pd.read_csv('2010-2012_hof_players.csv')
df_2010_2012

,Name,Position,Games,PPG,RPG,APG,PER,FG%,FT%,All-Stars,All-NBA,All-Defense,All-Rookie Team,MVPs,Chips,ROY,DPOYs,Scoring Champ,HOF
0,Mel Daniels,C,639,18.4,14.9,1.8,20.1,46.8,65.7,7,0,0,1,2,0,1,0,0,1
1,Reggie Miller,SG,1389,18.2,3.0,3.0,18.4,47.1,88.8,5,3,0,0,0,0,0,0,0,1
2,Ralph Sampson,C,456,15.4,8.8,2.3,16.0,48.6,66.1,4,1,0,1,0,0,1,0,0,1
3,Chet Walker,SF,1032,18.2,7.1,2.1,17.6,47.0,79.6,7,0,0,1,0,1,0,0,0,1
4,Jamaal Wilkes,SF,828,17.7,6.2,2.5,16.5,49.9,75.9,3,0,2,1,0,4,1,0,0,1
5,Artis Gilmore,C,1329,18.8,12.3,2.3,21.4,58.2,69.8,11,0,5,1,1,0,1,0,0,1
6,Chris Mullin,SF,986,18.2,4.1,3.5,18.8,50.9,86.5,5,4,0,0,0,0,0,0,0,1
7,Dennis Rodman,PF,911,7.3,13.1,1.8,14.6,52.1,58.4,2,2,8,0,0,5,0,2,0,1
8,Arvydas Sabonis,C,470,12.0,7.3,2.1,21.2,50.0,78.6,0,0,0,1,0,0,0,0,0,1
9,Dennis Johnson,PG,1100,14.1,3.9,5.0,14.6,44.5,79.7,5,2,9,0,0,3,0,0,0,1


In [4]:
df = pd.concat([df_2019_2013,df_2010_2012], ignore_index=True)
df

,Name,Position,Games,PPG,RPG,APG,PER,FG%,FT%,All-Stars,All-NBA,All-Defense,All-Rookie Team,MVPs,Chips,ROY,DPOYs,Scoring Champ,HOF
0,Carl Braun,PG,788,13.5,3.4,3.7,15.8,38.3,80.4,5,0,0,0,0,1,0,0,0,1
1,Chuck Cooper,SF,409,6.7,5.9,1.8,11.1,33.9,74.3,0,0,0,0,0,0,0,0,0,1
2,Vlade Divac,C,1134,11.8,8.2,3.1,17.7,49.5,69.2,1,0,0,1,0,0,0,0,0,1
3,Bobby Jones,PF,941,12.1,6.1,2.7,18.2,56.0,76.6,5,0,11,1,0,1,1,0,0,1
4,Sidney Moncrief,SG,767,15.6,4.7,3.6,18.7,50.2,83.1,5,5,5,0,0,0,0,2,0,1
5,Jack Sikma,C,1107,15.6,9.8,3.2,17.3,46.4,84.9,7,0,1,1,0,1,0,0,0,1
6,Paul Westphal,SG,823,15.6,1.9,4.4,19.4,50.4,82.0,5,4,0,0,0,1,0,0,0,1
7,Ray Allen,SG,1300,18.9,4.1,3.4,18.6,45.2,89.4,10,2,0,1,0,2,0,0,0,1
8,Maurice Cheeks,PG,1101,11.1,2.8,6.7,16.5,52.3,79.3,4,0,5,0,0,1,0,0,0,1
9,Grant Hill,SF,1026,16.7,6.0,4.1,19.0,48.3,76.9,7,5,0,1,0,0,1,0,0,1


Some players on here have awards equivalent to awards such as All-NBA or All-Star. For instance, the merger between the ABA (American Basketball Association) and the NBA (National Basketball Association) in 1976 brought over several players who earned All-ABA selections or participated in ABA All-Star Games. These achievements were highly regarded during their time and are comparable to their NBA counterparts. For this project, awards like All-ABA selections will be treated as equivalent to All-NBA honors, and ABA All-Star appearances will be considered equal to NBA All-Star appearances to maintain consistency across the dataset.


Similarly, awards such as All-BAA, like those earned by Carl Braun, will be treated as equivalent to modern All-NBA honors.

In [5]:
#Give players additional awards for NBA equivalents (All-BAA, All-ABA, ABA All-Star)
df.loc[0, 'All-NBA'] = 2 #Carl Braun had 2 All-BAA awards
df.loc[3, 'All-NBA'] = 1 #Bobby Jones had a All-ABA selection in 1975-76
df.loc[13, 'All-NBA'] = 2 #Charlie Scott had 2 All-ABA selections
df.loc[14, 'All-NBA'] = 5 #George McInnis has 3 All-ABA selections so I'll add that to his 2 All-NBA selections
df.loc[14, 'Chips'] = 2 #George McInnis also won 2 ABA Championships
df.loc[16, 'All-NBA'] = 2 #Zelmo Beaty has 2 All-ABA selections
df.loc[16, 'Chips'] = 1 #Zelmo Beaty also won a ABA championship
df.loc[20, 'All-NBA'] = 4 #Louie Dampier had 4 All-ABA selections
df.loc[20, 'Chips'] = 1 #Additionally Louie Dampier also had 1 ABA Championship
df.loc[21, 'All-NBA'] = 5 #Spencer Haywood had a All-ABA selections so I'll add that to his 4 All-NBA selections
df.loc[28, 'All-NBA'] = 3 #Roger brown had 3 All-ABA selections
df.loc[28, 'Chips'] = 3 #Roger brown also had 3 ABA Championships
df.loc[32, 'All-NBA'] = 5 #Mel Daniels had 5 All-ABA selections
df.loc[32, 'Chips'] = 3 #Mel Daniels had 3 ABA Championships
df.loc[37, 'All-NBA'] = 5 #Artis Gilmore had 5 All-ABA selections
df.loc[37, 'Chips'] = 1 #Artis Gilmore won a ABA championship
df.loc[42, 'Chips'] = 1 #Gus Johnson won a ABA championship


I'll need to update my code to include All-NBA or All-ABA players. I'll do the same for NBA or ABA championships.

In [6]:
df

,Name,Position,Games,PPG,RPG,APG,PER,FG%,FT%,All-Stars,All-NBA,All-Defense,All-Rookie Team,MVPs,Chips,ROY,DPOYs,Scoring Champ,HOF
0,Carl Braun,PG,788,13.5,3.4,3.7,15.8,38.3,80.4,5,2,0,0,0,1,0,0,0,1
1,Chuck Cooper,SF,409,6.7,5.9,1.8,11.1,33.9,74.3,0,0,0,0,0,0,0,0,0,1
2,Vlade Divac,C,1134,11.8,8.2,3.1,17.7,49.5,69.2,1,0,0,1,0,0,0,0,0,1
3,Bobby Jones,PF,941,12.1,6.1,2.7,18.2,56.0,76.6,5,1,11,1,0,1,1,0,0,1
4,Sidney Moncrief,SG,767,15.6,4.7,3.6,18.7,50.2,83.1,5,5,5,0,0,0,0,2,0,1
5,Jack Sikma,C,1107,15.6,9.8,3.2,17.3,46.4,84.9,7,0,1,1,0,1,0,0,0,1
6,Paul Westphal,SG,823,15.6,1.9,4.4,19.4,50.4,82.0,5,4,0,0,0,1,0,0,0,1
7,Ray Allen,SG,1300,18.9,4.1,3.4,18.6,45.2,89.4,10,2,0,1,0,2,0,0,0,1
8,Maurice Cheeks,PG,1101,11.1,2.8,6.7,16.5,52.3,79.3,4,0,5,0,0,1,0,0,0,1
9,Grant Hill,SF,1026,16.7,6.0,4.1,19.0,48.3,76.9,7,5,0,1,0,0,1,0,0,1


In [13]:
#Additionally, Bobby Jones has a ROY he shouldn't have, so I'll have to remove that
df.loc[3, 'ROY'] = 0

In [18]:
#Save final dataframe 
df.to_csv('2010s_hof_players.csv',index=False)